In [138]:
import os
import pandas as pd
from datetime import datetime, timedelta
import pandas_datareader as dr
import yfinance as yf
import numpy as np
import glob
from collections import Counter
from ast import literal_eval


In [139]:
# os.chdir('..')
# os.chdir('..')
cwd = os.getcwd()
print(cwd)

C:\Users\mfcli\Documents\Subreddit-Stock-Prediction


In [130]:
os.chdir('Users\mfcli\Documents\Subreddit-Stock-Prediction')

In [140]:
path = r'data' # use your path
all_files = glob.glob(path + "/*.jsonl")

tickCnt2 = Counter()

for filename in all_files:
    print(filename)
    df = pd.read_json(filename,lines=True)
    for x in df['tickers']:
        if len(x) > 0:
            y, z = x[0]
            tickCnt2[y] += 1

data\daytrading_comments.jsonl
data\daytrading_submission.jsonl
data\dividends_comments.jsonl
data\dividends_submission.jsonl
data\economy_comments.jsonl
data\economy_submission.jsonl
data\investing_comments.jsonl
data\investing_submission.jsonl
data\options_comments.jsonl
data\options_submission.jsonl
data\securityanalysis_comments.jsonl
data\securityanalysis_submission.jsonl
data\stockmarket_comments.jsonl
data\stockmarket_submission.jsonl
data\stocks_comments.jsonl
data\stocks_submission.jsonl


In [141]:
top_n = 11
top_list = tickCnt2.most_common(top_n)
print(top_list)

[('TSLA', 3382), ('AMD', 2875), ('AAPL', 2140), ('AMZN', 1981), ('MSFT', 1670), ('TD', 1597), ('GOOG', 1543), ('NIO', 1299), ('GE', 1281), ('FB', 1178), ('BABA', 1048)]


In [142]:
blacklist = ["TD"]
top_list = list(filter(lambda ticker_and_count: ticker_and_count[0] not in blacklist, top_list))
ticker_list = list(map(lambda t_a_c: t_a_c[0], top_list))

In [143]:
start = datetime(2014,1,1)
end = datetime(2021,1,1)

stock_data = {}
for ticker in ticker_list:
    stock = dr.data.DataReader(ticker, data_source='yahoo', start=start, end=end)
    stock_data[ticker] = stock

In [153]:
submission_sentiments = pd.read_csv("data/LabelledData/submissions_with_tickers_labelled.csv")
submission_sentiments.set_index("name", inplace=True)
comment_sentiments = pd.read_csv("data/LabelledData/comments_with_tickers_labelled.csv")

# Get Top
comment_sentiments["tickers"] = comment_sentiments["tickers"].apply(lambda x: literal_eval(str(x)))
comment_sentiments["ticker"] = comment_sentiments["tickers"].map(lambda tickers: tickers[0][0] if len(tickers) > 0 else np.NaN)
# print(ticker_list)
comment_sentiments = comment_sentiments[comment_sentiments["ticker"].isin(ticker_list)]
comment_sentiments.dropna()
# print(comment_sentiments["ticker"])

,Unnamed: 0,submission_name,id,text,score,created_utc,tickers,neg,neu,pos,ticker
1,1,t3_a4et2x,ebe85cf,Does anyone have a excel sample sheet they can...,3.0,1.544318e+09,"[[GOOG, 1]]",0.054193,0.913546,0.032261,GOOG
6,6,t3_an4n5g,efqslh9,Define cheap. For me I'm putting a bunch of mo...,3.0,1.549309e+09,"[[MSFT, 1]]",0.016944,0.317534,0.665522,MSFT
19,19,t3_bdmdvj,elfucsc,I am thinking about doing this strategy as wel...,2.0,1.555871e+09,"[[FB, 1]]",0.023970,0.929311,0.046719,FB
37,37,t3_cknban,evsth5n,MSFT TJX COST ADP APD HD AFL UNH VFC MKC,2.0,1.564756e+09,"[[MSFT, 1], [TJX, 1], [COST, 1], [ADP, 1], [AP...",0.019561,0.930602,0.049836,MSFT
55,55,t3_drbtts,f6h752b,MSFT and AAPL both pay dividends. You can set ...,8.0,1.572839e+09,"[[MSFT, 1], [AAPL, 1]]",0.017734,0.936916,0.045350,MSFT
...,...,...,...,...,...,...,...,...,...,...,...
41596,41596,t3_knnc52,ghm2c5k,Mgmt style difference.\n\nYou hear Adobe brag ...,3.0,1.609429e+09,"[[MSFT, 1]]",0.039295,0.924701,0.036005,MSFT
41597,41597,t3_knnc52,ghm73op,Azure will provide consistent growth for the n...,2.0,1.609431e+09,"[[MSFT, 2], [AMZN, 1]]",0.011626,0.846059,0.142315,MSFT
41598,41598,t3_knnc52,ghndxub,Ask yourself one question \n\nIs Microsoft off...,2.0,1.609454e+09,"[[MSFT, 1]]",0.014535,0.902674,0.082791,MSFT
41599,41599,t3_knnc52,ghlw67q,Tldr: MSFT to the moon 🚀🚀🚀🚀🚀🚀🚀,2.0,1.609425e+09,"[[MSFT, 1]]",0.018985,0.918247,0.062768,MSFT


In [154]:
comment_sentiments.head(20)

,Unnamed: 0,submission_name,id,text,score,created_utc,tickers,neg,neu,pos,ticker
1,1,t3_a4et2x,ebe85cf,Does anyone have a excel sample sheet they can...,3.0,1.544318e+09,"[[GOOG, 1]]",0.054193,0.913546,0.032261,GOOG
6,6,t3_an4n5g,efqslh9,Define cheap. For me I'm putting a bunch of mo...,3.0,1.549309e+09,"[[MSFT, 1]]",0.016944,0.317534,0.665522,MSFT
19,19,t3_bdmdvj,elfucsc,I am thinking about doing this strategy as wel...,2.0,1.555871e+09,"[[FB, 1]]",0.023970,0.929311,0.046719,FB
37,37,t3_cknban,evsth5n,MSFT TJX COST ADP APD HD AFL UNH VFC MKC,2.0,1.564756e+09,"[[MSFT, 1], [TJX, 1], [COST, 1], [ADP, 1], [AP...",0.019561,0.930602,0.049836,MSFT
55,55,t3_drbtts,f6h752b,MSFT and AAPL both pay dividends. You can set ...,8.0,1.572839e+09,"[[MSFT, 1], [AAPL, 1]]",0.017734,0.936916,0.045350,MSFT
71,71,t3_e6odtb,f9s2oel,So i use my own version of google sheet and si...,2.0,1.575585e+09,"[[GOOG, 2]]",0.029604,0.943171,0.027225,GOOG
124,124,t3_ffvdz5,fk3ij8k,i think oxy dividend is just not sustainable \...,3.0,1.583835e+09,"[[TSLA, 1]]",0.710343,0.263356,0.026301,TSLA
154,154,t3_fq0pgd,flo705m,The annual income for MSFT and other stocks ar...,2.0,1.585333e+09,"[[MSFT, 1]]",0.019378,0.945318,0.035303,MSFT
191,191,t3_gdronb,fpkokle,"First, you are not late. Get those negative th...",3.0,1.588698e+09,"[[MSFT, 1]]",0.029374,0.914952,0.055674,MSFT
196,196,t3_gf13af,fpqx899,MSFT is one of only two companies with an AAA ...,18.0,1.588835e+09,"[[MSFT, 1]]",0.007829,0.592571,0.399600,MSFT


In [155]:
submission_sentiments

,Unnamed: 0,title,text,score,upvote_ratio,created_utc,tickers,neg,neu,pos
name,,,,,,,,,,
t3_1ubci0,0,Help with analyzing telecoms companies,"Hello Security Analysts,\nI am currently doing...",9.0,0.86,1.388763e+09,"[['DCF', 1]]",0.714837,0.243861,0.041302
t3_1uofbz,1,10-K Club Round 2: DE (Deere and Company),**Goal**: To form a buy/sell/hold recommendati...,28.0,0.83,1.389149e+09,"[['DE', 1], ['NEXT', 1], ['CAP', 1]]",0.014544,0.930853,0.054602
t3_1vvhry,2,Thoughts on Declining Revenues? (Pfizer),"Hello everyone,\n\n\nWhat is your thoughts on ...",4.0,1.00,1.390419e+09,"[['PEG', 2], ['ZTS', 1]]",0.637652,0.309594,0.052755
t3_1wkj6b,3,How Accurate Are DCF Models?,"Hi guys, I wanted to start a discussion about ...",15.0,0.81,1.391103e+09,"[['DCF', 3]]",0.019967,0.917189,0.062844
t3_1xpps3,4,How to calculate intrinsic value,One of the things that great value investors (...,9.0,0.91,1.392220e+09,"[['BV', 1]]",0.020951,0.886475,0.092574
...,...,...,...,...,...,...,...,...,...,...
t3_kngcvy,40316,TSLA calls before deliveries announcement 1/4/...,"Alright, so Tesla popped today since an analys...",3.0,0.71,1.609380e+09,"[['TSLA', 1]]",0.030445,0.754021,0.215533
t3_knfd7d,40317,My Watchlist For 12/31/2020,My Plays for 12/31/2020:\n\n&#x200B;\n\nWhat e...,4.0,0.65,1.609377e+09,"[['RSI', 5]]",0.028625,0.903874,0.067501
t3_knekut,40318,AIV Stock Dividend,"I’ve done some research, and see how their div...",2.0,0.67,1.609374e+09,"[['AIV', 2]]",0.217989,0.748646,0.033365


In [25]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
  
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
config = AutoConfig.from_pretrained("ProsusAI/finbert", output_hidden_states=True)
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert", config=config)

import torch

In [43]:
config

BertConfig {
  "_name_or_path": "ProsusAI/finbert",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "positive",
    "1": "negative",
    "2": "neutral"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 1,
    "neutral": 2,
    "positive": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.5.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [95]:
# For submissions
submission_embeddings = []
for index, row in submission_sentiments.iterrows():
    text = row["title"] + row["text"]
    inputs = tokenizer.encode_plus(text,add_special_tokens=True, truncation=True, return_tensors="pt")
    outputs = model(**inputs)
    embeddings = outputs.hidden_states[1:] # tuple of length 12 (for each layer)
    # Each tensor in embeddings is of the shape – (num_samples x max_length x hidden_size)
    avg_embeddings = torch.mean(torch.stack(embeddings), dim=0).squeeze() #taking the avg of tensors from all 12 layers
    # getting embeddings for the TICKER
    ticker = "".join(list(row["tickers"])[3:6]).lower()
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    idx = 0
    for i, token in enumerate(tokens):
        if token == ticker or token == ticker[0:1]:
            idx = i
            break
    submission_embeddings.append(avg_embeddings[idx, :].detach().numpy())
        

In [149]:
#submission_embeddings_df = pd.DataFrame(dict(embeddings=np.loadtxt(r"data/LabelledData/submissions_sentiments_embeddings.csv",delimiter=",")))

In [165]:
np.loadtxt(r"data/LabelledData/submissions_sentiments_embeddings.csv",delimiter=",")

array([[ 0.25317159, -0.33728263, -0.32491115, ..., -0.06689958,
         0.1287805 ,  0.23185521],
       [ 0.11967877, -0.47373697, -0.24815078, ..., -0.15162687,
         0.06769352,  0.35467413],
       [-0.10520971, -0.35760066,  0.13364257, ..., -0.44605622,
         0.69378448,  0.95332414],
       ...,
       [ 0.25676253, -0.40016314, -0.26890647, ..., -0.04746638,
         0.10861386,  0.24735175],
       [ 0.44633231,  0.33704826, -0.24836381, ..., -0.34289292,
         1.28146899,  0.12751247],
       [ 0.33859858, -0.38206935, -0.05458961, ..., -0.05624842,
         0.18331234,  0.15550326]])

In [148]:
submission_embeddings_df

,embeddings
0,"[0.2531716, -0.33728263, -0.32491115, -0.21769..."
1,"[0.11967877, -0.47373697, -0.24815078, -0.3391..."
2,"[-0.10520971, -0.35760066, 0.13364257, -0.1795..."
3,"[0.3095707, -0.3475566, -0.21459417, -0.295747..."
4,"[0.7541446, 0.08881012, 0.09770101, -0.6020092..."
...,...
40316,"[0.2894551, -0.57394236, -0.09654113, -0.32213..."
40317,"[0.2779404, -0.5043637, -0.03495891, -0.207007..."
40318,"[0.25676253, -0.40016314, -0.26890647, -0.2777..."
40319,"[0.4463323, 0.33704826, -0.24836381, -0.723890..."


In [156]:
submission_sentiments.reset_index(drop=False, inplace=True)
submission_embeddings_df.reset_index(drop=True, inplace=True)
dfs = [submission_sentiments, submission_embeddings_df]
submission_sentiments_embeddings = pd.concat(dfs, axis=1)

In [114]:
np.savetxt(r"data/LabelledData/submissions_sentiments_embeddings.csv",submission_embeddings,delimiter=",")

In [158]:
submission_sentiments_embeddings.set_index("name", inplace=True)
submission_sentiments.set_index("name", inplace=True)


In [ ]:
# For comments
comments_embeddings = []
for index, row in comment_sentiments.iterrows():
    text = row["text"]
    inputs = tokenizer.encode_plus(text,add_special_tokens=True, truncation=True, return_tensors="pt")
    outputs = model(**inputs)
    embeddings = outputs.hidden_states[1:] # tuple of length 12 (for each layer)
    # Each tensor in embeddings is of the shape – (num_samples x max_length x hidden_size)
    avg_embeddings = torch.mean(torch.stack(embeddings), dim=0).squeeze() #taking the avg of tensors from all 12 layers
    # getting embeddings for the TICKER
    ticker = "".join(list(row["tickers"])[3:6]).lower()
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    idx = 0
    for i, token in enumerate(tokens):
        if token == ticker or token == ticker[0:1]:
            idx = i
            break
    submission_embeddings.append(avg_embeddings[idx, :].detach().numpy())
    print(index)
        

In [ ]:
comments_embeddings_df = pd.DataFrame(dict(embeddings=comments_embeddings))

In [ ]:
submission_sentiments.reset_index(drop=True, inplace=True)
submission_embeddings_df.reset_index(drop=True, inplace=True)
dfs = [comment_sentiments, comments_embeddings_df]
comment_sentiments_embeddings = pd.concat(dfs, axis=1)

In [ ]:
comment_sentiments_embeddings.to_csv("../../data/LabelledData/comment_sentiments_embeddings.csv")
comment_sentiments_embeddings.to_json("../../data/LabelledData/comment_sentiments_embeddings.csv", lines=True, orient='record')

In [184]:
avg_t_span = timedelta(weeks=1)
span_interval = timedelta(days=1)
future_t_interval = timedelta(days=3)

for filename in all_files:
    if not filename.endswith("submission.jsonl"):
        continue

    subreddit_path = filename.replace("_submission.jsonl", "")
    print(filename)
    
    df = pd.read_json(filename,lines=True)
    df["date"] = pd.to_datetime(df['created_utc'],unit='s')
    
    # clean dataset
    df["ticker"] = df["tickers"].map(lambda tickers: tickers[0][0] if len(tickers) > 0 else np.NaN)
    df = df[df["ticker"].isin(ticker_list)]
    df.dropna()
    df.set_index("date", inplace=True)
    df.sort_index(inplace=True)
    
    features = []
    
    t = start
    while t <= end - avg_t_span:
#         mask = (df >= t) & (df <= t + avg_t_span)
#         submissions = df.loc[mask]
#         print(df.index)
#         print(t)
        if(t.year != (t + avg_t_span).year):
            print(t)

        grouped_submissions = df[str(t):str(t+avg_t_span)]
        if len(grouped_submissions) == 0:
            t += span_interval
            continue
        
        comment_mask = comment_sentiments["submission_name"].isin(grouped_submissions["name"])
        comment_mask = comment_mask & comment_sentiments["ticker"].isin(ticker_list)
        grouped_comments = comment_sentiments.loc[comment_mask]
        
        for ticker in ticker_list:
            submissions = grouped_submissions[grouped_submissions["ticker"] == ticker]
            if len(submissions) == 0:
                continue
            
            comment_mask = grouped_comments["submission_name"].isin(submissions["name"])
#             comment_mask = comment_mask & (grouped_comments["ticker"] == ticker)
            comments = grouped_comments.loc[comment_mask]
            
            s_scores = submissions["score"].values
            s_uv_ratio = submissions["upvote_ratio"].values

            s_sentiments = submission_sentiments.loc[submissions["name"],["neg","neu","pos"]].values
            s_sentiments *= (s_scores * s_uv_ratio).reshape(-1,1)
            s_sentiments = s_sentiments.sum(axis=0)

            if len(comments) == 0:
                c_sentiments = np.zeros(3)
            else:
                c_sentiments = comments[["neg","neu","pos"]].values
                c_scores = comments["score"].values
                c_sentiments = (c_sentiments * c_scores.reshape(-1,1)).sum(axis=0)
                
            # Get Stock Stuff
            stock = stock_data[ticker]
            cur_stock_idx = stock.index.get_loc(t, method="nearest")
            future_stock_idx = stock.index.get_loc(t + avg_t_span, method="nearest")
            cur_stock = stock.iloc[cur_stock_idx]["Open"] #Note : is rounded to nearest date
            future_stock = stock.iloc[future_stock_idx]["Close"]
            
            label = 1 if cur_stock <= future_stock else 0
            
#             feature_vector = np.concatenate((s_sentiments, np.array(submission_sentiments_embeddings.loc[submissions["name"],["embeddings"]])[0],c_sentiments, np.array([label])), axis=0)
            feature_vector = s_sentiments.tolist() + np.array(submission_sentiments_embeddings.loc[submissions["name"],["embeddings"]])[0][0].tolist() + c_sentiments.tolist() + [label]
#             print(np.array(submission_sentiments_embeddings.loc[submissions["name"],["embeddings"]])[0][0].tolist())
#             print(feature_vector)
            features.append(feature_vector)
            
        
        t += span_interval
    pd.DataFrame(features).to_csv(f"{subreddit_path}_features_labels.csv", index=False)
    


data\daytrading_submission.jsonl
2014-12-25 00:00:00
2014-12-26 00:00:00
2014-12-27 00:00:00
2014-12-28 00:00:00
2014-12-29 00:00:00
2014-12-30 00:00:00
2014-12-31 00:00:00
2015-12-25 00:00:00
2015-12-26 00:00:00
2015-12-27 00:00:00
2015-12-28 00:00:00
2015-12-29 00:00:00
2015-12-30 00:00:00
2015-12-31 00:00:00
2016-12-25 00:00:00
2016-12-26 00:00:00
2016-12-27 00:00:00
2016-12-28 00:00:00
2016-12-29 00:00:00
2016-12-30 00:00:00
2016-12-31 00:00:00
2017-12-25 00:00:00
2017-12-26 00:00:00
2017-12-27 00:00:00
2017-12-28 00:00:00
2017-12-29 00:00:00
2017-12-30 00:00:00
2017-12-31 00:00:00
2018-12-25 00:00:00
2018-12-26 00:00:00
2018-12-27 00:00:00
2018-12-28 00:00:00
2018-12-29 00:00:00
2018-12-30 00:00:00
2018-12-31 00:00:00
2019-12-25 00:00:00
2019-12-26 00:00:00
2019-12-27 00:00:00
2019-12-28 00:00:00
2019-12-29 00:00:00
2019-12-30 00:00:00
2019-12-31 00:00:00
2020-12-25 00:00:00
data\dividends_submission.jsonl
2014-12-25 00:00:00
2014-12-26 00:00:00
2014-12-27 00:00:00
2014-12-28 00:0

In [177]:
np.array(submission_sentiments_embeddings.loc["t3_1ubci0",["embeddings"]])[0].tolist()

[0.2531715929508209,
 -0.33728262782096863,
 -0.3249111473560333,
 -0.2176937609910965,
 0.11065670102834702,
 -0.1066456213593483,
 0.0355328731238842,
 0.21947932243347168,
 0.06902558356523514,
 -0.531270444393158,
 -0.031090328469872475,
 0.18411247432231903,
 -0.054895978420972824,
 0.22180859744548798,
 -0.2924477458000183,
 0.09679508954286575,
 -0.20550091564655304,
 0.2737608253955841,
 0.053467895835638046,
 0.0028178293723613024,
 -0.4130427837371826,
 -0.2685052156448364,
 0.15470467507839203,
 0.1431085467338562,
 0.08916601538658142,
 -0.1658211201429367,
 -0.19471502304077148,
 0.041800323873758316,
 -0.3809865713119507,
 0.24446213245391846,
 -0.16805694997310638,
 0.30518022179603577,
 -0.23185332119464874,
 -0.3540884554386139,
 0.13372457027435303,
 -0.142230823636055,
 0.2846316397190094,
 0.08345342427492142,
 0.02445124089717865,
 -0.020283978432416916,
 0.0007818589801900089,
 0.1006108820438385,
 0.05413659289479256,
 -0.27546370029449463,
 -0.08129248768091202,

In [170]:
[1,2] + [3,4]

[1, 2, 3, 4]